In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import os
import random

In [3]:
dirr = 'osti.gov_13670'
result_log = json.load(open(f'writes/{dirr}/result_log.json', 'r'))
log = result_log['log'][1]
updated_code = log['updated']
with open(f'test/overrided.js', 'w+') as f:
    f.write(updated_code)

### Count uncaught exception and syntax error

In [6]:
total = 0
uncaught = []
syntax = []
dirs = os.listdir('writes')
for dirr in dirs:
    if not os.path.exists(f'writes/{dirr}/result_log.json'):
        continue
    result_log = json.load(open(f'writes/{dirr}/result_log.json', 'r'))
    log = result_log['log'][0]
    total += 1
    uncaught.append(log['uncaught'])
    if log['uncaught'] > 0:
        print(dirr)
    syntax.append(log['syntaxError'])
print(len(uncaught), len([x for x in uncaught if x > 0]))
print(len(syntax), len([x for x in syntax if x > 0]))

search.dea.gov_6220
internal.nren.nasa.gov_6006
myssaisanfrancisco.usajobs.gov_7717
diversitynews.msfc.nasa.gov_6527
www.uscourts.gov_1150
martin.floridahealth.gov_13234
investors.maine.gov_9477
newhaven.jobcorps.gov_10578
lni.wa.gov_12080
bve.ky.gov_4027
www.supremecourt.gov_1215
in.gov_7907
mpr.ams.usda.gov_3156
whigham.georgia.gov_7129
afisr.af.mil_6450
195wg.ang.af.mil_11356
odum.georgia.gov_11194
public.govdelivery.com_4953
ircalc.usps.gov_9452
ihs.gov_13622
lance.house.gov_1458
ots.gov_5737
usafacademyband.af.mil_8934
ehealth.ky.gov_4117
economist.uat.usajobs.gov_5943
siren.illinois.gov_3098
guam.gov_7606
flickr.com_7265
kathleenrice.house.gov_4366
nccpaboard.gov_11657
info.usitc.gov_12136
usgovernment.gov_9914
ssaiseattle.usajobs.gov_8322
miami.va.gov_5576
librarycatalog.dol.gov_2133
rader.nrmc.amedd.army.mil_6913
youtube.com_5799
testwater.weather.gov_3006
spanish.malabo.usembassy.gov_11513
medicalsurgical.va.gov_8742
azsos.gov_2159
guam.gov_8185
plus.google.com_8518
suicidepre

### Collect archive URLs for eot-1k

In [4]:
data = json.load(open('../../fidelity-files/metadata/eot-1k_metadata.json', 'r'))
data_list = [{'hostname': v['directory'], 'archive_url': v['archive']} for v in data.values()]
data_list = random.sample(data_list, 200)
json.dump(data_list, open('eot_1k_sampled_200.json', 'w+'), indent=2)

### Check wayback neighbor URLs

In [5]:
import requests
import time

def wayback_index(url, param_dict={}, wait=True, total_link=False, proxies={}):
    """00
    Get the wayback machine index of certain url by querying the CDX
    wait: wait unitl not getting block
    total_link: Returned url are in full(wayback) links

    return: ( [(timestamp, url, stauts_code)], SUCCESS/EMPTY/ERROR_MSG)
    """
    wayback_home = 'http://web.archive.org/web/'
    params = {
        'output': 'json',
        'url': url,
        'from': 19700101,
        'to': 20241231,
    }   
    params.update(param_dict)
    count = 0
    r = None
    while True:
        try:
            r = requests.get('http://web.archive.org/cdx/search/cdx', params=params, proxies=proxies, timeout=120)
            r = r.json()
            time.sleep(0.5)
            break
        except requests.exceptions.ConnectionError as e:
            time.sleep(20)
            continue
        except Exception as e:
            error_msg = str(e).split('\n')[0]
            if not r or not wait or r.status_code not in [429, 445, 501, 503]:
                return [], str(e)
            if count > 3:
                return [], str(e)
            count += 1
            time.sleep(10)
    if total_link:
        r = [(i[1], f"{wayback_home}{i[1]}/{i[2]}", i[4]) for i in r[1:]]
    else:
        r = [(i[1], i[2], i[4]) for i in r[1:]]
    if len(r) != 0:
        return r, "Success",
    else:
        return [], "Empty"

prefix = 'https://www.azsos.gov/sites/all/modules/ldap/ldap_user/ldap_user.css*'
param_dict = {
    'filter': ['statuscode:200'],
    'collapse': 'digest'
}

wi, status = wayback_index(prefix, param_dict=param_dict, total_link=True)

In [7]:
wi

[('20161115145359',
  'http://web.archive.org/web/20161115145359/https://www.azsos.gov/sites/all/modules/ldap/ldap_user/ldap_user.css',
  '200'),
 ('20161116220459',
  'http://web.archive.org/web/20161116220459/https://www.azsos.gov/sites/all/modules/ldap/ldap_user/ldap_user.css?ogn7oe',
  '200'),
 ('20161118050600',
  'http://web.archive.org/web/20161118050600/https://www.azsos.gov/sites/all/modules/ldap/ldap_user/ldap_user.css?ogt0v0',
  '200'),
 ('20170227204442',
  'http://web.archive.org/web/20170227204442/https://www.azsos.gov/sites/all/modules/ldap/ldap_user/ldap_user.css?olwjmc',
  '200'),
 ('20170308144340',
  'http://web.archive.org/web/20170308144340/https://www.azsos.gov/sites/all/modules/ldap/ldap_user/ldap_user.css?om70go',
  '200')]